In [1]:
from datasets import load_dataset

In [2]:
dataset_train = load_dataset('ms_marco', 'v2.1', split='train')

In [3]:
dataset_train

Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 808731
})

In [4]:
def preprocess(example):
    positive_passages=[]
    for p in enumerate(example['passages']["is_selected"]):
        if p[1]==1:
            positive_passages.append(example["passages"]["passage_text"][p[0]])
    negative_passages = []
    for p in enumerate(example['passages']["is_selected"]):
        if p[1]==0:
            negative_passages.append(example["passages"]["passage_text"][p[0]])
    if (len(positive_passages)>0 and len(negative_passages)>=5):
        positive = positive_passages[0]
        negatives = negative_passages[:5]
        return {
            "query": example["query"],
            "positive": positive,
            "negatives": negatives
        }
    else:
        return {"query": None, "positive": None, "negatives": None}

In [5]:
processed_data_train = dataset_train.map(preprocess, remove_columns=dataset_train.column_names)

In [6]:
processed_data_train = processed_data_train.filter(lambda x: x['query'] is not None and x['positive'] is not None)

In [7]:
contrastive_pairs_train = []
for item in processed_data_train:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_train.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [8]:
len(contrastive_pairs_train)

502824

In [9]:
from torch.utils.data import DataLoader

In [10]:
class ContrastiveDataset:
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]
        return item["anchor"], item["positive"], item["negatives"]

In [11]:
contrastive_dataset_train = ContrastiveDataset(contrastive_pairs_train)

In [12]:
data_loader_train = DataLoader(contrastive_dataset_train, batch_size=32, shuffle=True)

In [13]:
len(data_loader_train)

15714

In [14]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [15]:
model = AutoModel.from_pretrained("bert-base-uncased")

In [16]:
from peft import LoraConfig, get_peft_model

In [17]:
lora_config = LoraConfig(
    task_type= "FEATURE_EXTRACTION"
)

In [18]:
lora_model = get_peft_model(model, lora_config)

In [19]:
import torch

In [20]:
def cosine_distance(x, y):
    return 1 - torch.nn.functional.cosine_similarity(x, y, dim=-1)

In [21]:
def info_nce_loss(anchor_embedding, positive_embedding, negative_embedding, distance_fn):

    pos_dist = distance_fn(anchor_embedding, positive_embedding) 
    neg_dist = torch.stack([distance_fn(anchor_embedding, neg) for neg in negative_embedding], dim=-1) 

    logits = torch.cat([-pos_dist.unsqueeze(1), -neg_dist], dim=1)
    labels = torch.zeros(logits.size(0), dtype=torch.long, device=logits.device)

    loss = torch.nn.CrossEntropyLoss()(logits, labels)
    return loss

In [22]:
import torch.optim as optim
optimizer = torch.optim.AdamW(lora_model.parameters(), lr=5e-5)

In [23]:
num_epochs=3

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
device

device(type='cuda')

In [26]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.5.1+cu124
True


In [27]:
torch.cuda.is_available()

True

In [28]:
lora_model = lora_model.to(device)

In [29]:
dataset_val = load_dataset('ms_marco', 'v2.1', split='validation')

In [30]:
processed_data_val = dataset_val.map(preprocess, remove_columns=dataset_val.column_names)

In [31]:
processed_data_val = processed_data_val.filter(lambda x: x['query'] is not None and x['positive'] is not None)

In [32]:
contrastive_pairs_val = []
for item in processed_data_val:
    query = item["query"]
    positive = item["positive"]
    negatives = item["negatives"]
    contrastive_pairs_val.append({
        "anchor": query,
        "positive": positive,
        "negatives": negatives
    })

In [33]:
contrastive_dataset_val = ContrastiveDataset(contrastive_pairs_val)

In [34]:
data_loader_val = DataLoader(contrastive_dataset_val, batch_size=32, shuffle=True)

In [35]:
len(data_loader_val)

1737

In [36]:
def evaluate_mrr(model, data_loader_val, distance_fn):
    model.eval()

    total_rr = 0.0
    num_queries = 0

    with torch.no_grad():
        for batch in data_loader_val:
            anchor_text = batch[0]
            positive_text = batch[1]
            negative_texts = batch[2]

            anchor_input = tokenizer(anchor_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
            positive_input = tokenizer(positive_text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

        
            anchor_embedding = model(**anchor_input).last_hidden_state[:, 0, :]
            positive_embedding = model(**positive_input).last_hidden_state[:, 0, :]
            negative_embedding = [model(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :] for neg in negative_texts]

            pos_dist = distance_fn(anchor_embedding, positive_embedding)
            neg_dist = torch.stack([distance_fn(anchor_embedding, neg) for neg in negative_embedding], dim=-1)
            all_similarities=torch.cat([-pos_dist.unsqueeze(1), -neg_dist], dim=1)

            sorted_similarities, sorted_indices = torch.sort(all_similarities, dim=1, descending=True)

            # Find the rank of the first relevant (positive) document
            positive_rank = (sorted_indices == 0).nonzero(as_tuple=True)[1] + 1  # +1 to make rank 1-based
            total_rr += torch.sum(1.0 / positive_rank.float()).item()
            num_queries += len(positive_rank)

    mrr = total_rr / num_queries
    return mrr

In [37]:
import os
save_dir ="/dss/dsshome1/07/ra65bex2/srawat/v2/baseline"

In [38]:
import time
epoch_metrics = []

In [39]:
for epoch in range(num_epochs):
    start_time = time.time()
    lora_model.train() 
    
    total_loss = 0.0
    for batch in data_loader_train:
    
        anchor_texts = batch[0]
        positive_texts = batch[1]
        negative_texts = batch[2]

        anchor_inputs = tokenizer(anchor_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        positive_inputs = tokenizer(positive_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    
        anchor_embedding = lora_model(**anchor_inputs).last_hidden_state[:, 0, :]
        positive_embedding = lora_model(**positive_inputs).last_hidden_state[:, 0, :]
        negative_embedding = [lora_model(**tokenizer(neg, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)).last_hidden_state[:, 0, :] for neg in negative_texts]

        loss = info_nce_loss(anchor_embedding, positive_embedding, negative_embedding, distance_fn=cosine_distance)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    save_path = os.path.join(save_dir, f"checkpoint_epoch_{epoch+1}.pth")
    torch.save(lora_model, save_path)
    print(f"EPOCH {epoch+1}:")
    print(f"Checkpoint saved: {save_path}")
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_loss / len(data_loader_train)}")
    mrr_validation = evaluate_mrr(lora_model, data_loader_val, cosine_distance)
    #mrr_train = evaluate_mrr(lora_model, data_loader_train, cosine_distance)
    #print(f"Mean Reciprocal Rank (MRR) for training set: {mrr_train:.4f}")
    print(f"Mean Reciprocal Rank (MRR) for validation set: {mrr_validation:.4f}")
    end_time = time.time()
    print(f"Epoch {epoch+1} took {(end_time - start_time) / 60:.4f} minutes.")
    print(f"\n")
    epoch_metrics.append({
        'epoch': epoch + 1,
        'training_loss': total_loss / len(data_loader_train),
        'mrr_validation': mrr_validation,
        'time_taken_minutes': (end_time - start_time) / 60
    })

EPOCH 1:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/v2/baseline/checkpoint_epoch_1.pth
Epoch 1/3, Training Loss: 1.585928325815724


Mean Reciprocal Rank (MRR) for validation set: 0.6409
Epoch 1 took 217.5235 minutes.




EPOCH 2:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/v2/baseline/checkpoint_epoch_2.pth
Epoch 2/3, Training Loss: 1.5439890113452834


Mean Reciprocal Rank (MRR) for validation set: 0.6474
Epoch 2 took 217.3832 minutes.




EPOCH 3:
Checkpoint saved: /dss/dsshome1/07/ra65bex2/srawat/v2/baseline/checkpoint_epoch_3.pth
Epoch 3/3, Training Loss: 1.5308515597758319


Mean Reciprocal Rank (MRR) for validation set: 0.6533
Epoch 3 took 217.5316 minutes.




In [40]:
import json

In [41]:
with open(save_dir + '/epoch_metrics.json', 'w') as f:
    json.dump(epoch_metrics, f)